In [32]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd

In [33]:
import psycopg2
from psycopg2 import OperationalError
# connect to the server
def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection

In [34]:
# Connection details for OLTP and OLAP databases
oltp_db_name = "olympicsOLTP"
oltp_db_user = "postgres"
oltp_db_password = "postgres"
oltp_db_host = "pgdb"
oltp_db_port = "5432"

olap_db_name = "olympicsOLAP"
olap_db_user = "postgres"
olap_db_password = "postgres"
olap_db_host = "pgdb"
olap_db_port = "5432"

# Create connections to OLTP and OLAP databases
oltp_connection = create_connection(oltp_db_name, oltp_db_user, oltp_db_password, oltp_db_host, oltp_db_port)
olap_connection = create_connection(olap_db_name, olap_db_user, olap_db_password, olap_db_host, olap_db_port)
oltp_connection_url = f"postgresql://{oltp_db_user}:{oltp_db_password}@{oltp_db_host}:{oltp_db_port}/{oltp_db_name}"
olap_connection_url = f"postgresql://{olap_db_user}:{olap_db_password}@{olap_db_host}:{olap_db_port}/{olap_db_name}"
oltp_engine = create_engine(oltp_connection_url)
olap_engine = create_engine(olap_connection_url)

Connection to PostgreSQL DB successful
Connection to PostgreSQL DB successful


In [35]:
factolympicmedals = pd.read_sql('SELECT * FROM factolympicmedals', con=olap_engine)
dimeventcategory = pd.read_sql('SELECT * FROM dimeventcategory', con=olap_engine)
dimgamehost = pd.read_sql('SELECT * FROM dimgamehost', con=olap_engine)
dimcountry = pd.read_sql('SELECT * FROM dimcountry', con=olap_engine)
merged_data = pd.merge(factolympicmedals, dimeventcategory, on='eventcategorykey')
merged_data = pd.merge(merged_data, dimgamehost, left_on='hostkey', right_on='gamehostkey')
merged_data = pd.merge(merged_data, dimcountry, on='countrykey')

In [36]:
merged_data

,countrykey,hostkey,medaltype,eventattendtypekey,eventcategorykey,timekey,populationkey,lifeexpectancykey,mentalillnesskey,gdpkey,eventdiscipline,eventname,gamehostkey,gamehostname,gamehostcity,gamehostseasontype,gamehostcountry,countryalternativekey,countryname,countryregion
0,25,1,GOLD,1,1,YR2022,NaN,NaN,NaN,NaN,Curling,Mixed Doubles,1,Beijing 2022,Beijing,Winter,China,ITA,Italy,Europe
1,25,1,GOLD,1,1,YR2022,NaN,NaN,NaN,NaN,Curling,Mixed Doubles,1,Beijing 2022,Beijing,Winter,China,ITA,Italy,Europe
2,25,1,GOLD,5,17,YR2022,NaN,NaN,NaN,NaN,Short Track Speed Skating,Women's 500m,1,Beijing 2022,Beijing,Winter,China,ITA,Italy,Europe
3,25,1,BRONZE,3,19,YR2022,NaN,NaN,NaN,NaN,Short Track Speed Skating,Men's 5000m Relay,1,Beijing 2022,Beijing,Winter,China,ITA,Italy,Europe
4,25,1,SILVER,1,20,YR2022,NaN,NaN,NaN,NaN,Short Track Speed Skating,Team Relay,1,Beijing 2022,Beijing,Winter,China,ITA,Italy,Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21490,210,52,BRONZE,5,836,YR1900,NaN,NaN,NaN,NaN,Tennis,Singles women,52,Paris 1900,Paris,Summer,France,BOH,Bohemia,Europe
21491,31,8,SILVER,4,588,YR2008,10.0,4.0,9.0,8.0,Athletics,800m men,8,Beijing 2008,Beijing,Summer,China,SDN,Sudan,Africa
21492,177,8,SILVER,5,644,YR2008,2.0,7.0,3.0,2.0,Weightlifting,+75kg women,8,Beijing 2008,Beijing,Summer,China,WSM,Samoa,Oceania
21493,98,8,BRONZE,4,939,YR2008,6.0,3.0,7.0,4.0,Canoe Slalom,K1 kayak single men,8,Beijing 2008,Beijing,Summer,China,TGO,Togo,Africa


In [37]:
merged_data['country_year'] = merged_data['countryname'] + '_' + merged_data['gamehostname']

In [38]:
merged_data.isna().sum()

countrykey                   0
hostkey                      0
medaltype                    0
eventattendtypekey           0
eventcategorykey             0
timekey                      0
populationkey            10940
lifeexpectancykey         5828
mentalillnesskey         12623
gdpkey                    7747
eventdiscipline              0
eventname                    0
gamehostkey                  0
gamehostname                 0
gamehostcity                 0
gamehostseasontype           0
gamehostcountry              0
countryalternativekey        0
countryname                  0
countryregion                0
country_year                 0
dtype: int64

In [39]:
merged_data = merged_data.astype(str)

In [40]:
merged_data = merged_data[['eventdiscipline', 'country_year']]
merged_data = merged_data.drop_duplicates()

In [41]:
merged_data

,eventdiscipline,country_year
0,Curling,Italy_Beijing 2022
2,Short Track Speed Skating,Italy_Beijing 2022
6,Snowboard,Italy_Beijing 2022
9,Luge,Italy_Beijing 2022
10,Biathlon,Italy_Beijing 2022
...,...,...
21490,Tennis,Bohemia_Paris 1900
21491,Athletics,Sudan_Beijing 2008
21492,Weightlifting,Samoa_Beijing 2008
21493,Canoe Slalom,Togo_Beijing 2008


In [42]:
grouped = merged_data.groupby('country_year')['eventdiscipline'].apply(list).reset_index()
grouped

,country_year,eventdiscipline
0,Afghanistan_Beijing 2008,[Taekwondo]
1,Afghanistan_London 2012,[Taekwondo]
2,Algeria_Atlanta 1996,"[Athletics, Boxing]"
3,Algeria_Barcelona 1992,"[Athletics, Boxing]"
4,Algeria_Beijing 2008,[Judo]
...,...,...
1748,Zambia_Atlanta 1996,[Athletics]
1749,Zambia_Los Angeles 1984,[Boxing]
1750,Zimbabwe_Athens 2004,[Swimming]
1751,Zimbabwe_Beijing 2008,[Swimming]


In [43]:
from mlxtend.preprocessing import TransactionEncoder

In [44]:
# thx for GPT 4 as below transformation
te = TransactionEncoder()
te_ary = te.fit_transform(grouped['eventdiscipline'])
transformed_df = pd.DataFrame(te_ary, columns=te.columns_)

In [45]:
transformed_df

,3x3 Basketball,Alpine Skiing,Archery,Artistic Gymnastics,Artistic Swimming,Athletics,Badminton,Baseball,Baseball/Softball,Basketball,...,Tennis,Trampoline,Trampoline Gymnastics,Triathlon,Tug of War,Volleyball,Water Motorsports,Water Polo,Weightlifting,Wrestling
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1749,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1750,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1751,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [46]:
from mlxtend.frequent_patterns import apriori, association_rules

In [48]:
#setting threshold
frequent_itemsets = apriori(transformed_df, min_support=0.04, use_colnames=True, max_len=6)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.90)
rules = rules.sort_values(by='lift', ascending=False)

print(rules)

                                 antecedents  consequents  antecedent support  \
21               (Rowing, Diving, Athletics)   (Swimming)            0.045636   
6                           (Rowing, Diving)   (Swimming)            0.047918   
28         (Boxing, Shooting, Weightlifting)  (Wrestling)            0.048488   
30     (Boxing, Rowing, Swimming, Wrestling)  (Athletics)            0.040502   
0                           (Boxing, Diving)  (Athletics)            0.042784   
3                         (Diving, Swimming)  (Athletics)            0.052481   
2                         (Diving, Shooting)  (Athletics)            0.045066   
20                (Rowing, Diving, Swimming)  (Athletics)            0.043354   
26             (Rowing, Swimming, Wrestling)  (Athletics)            0.051911   
14                (Boxing, Rowing, Swimming)  (Athletics)            0.056475   
25              (Swimming, Rowing, Shooting)  (Athletics)            0.050770   
12                 (Sailing,